# Practice Learning Activity 5 (Evaluate models on use cases and for safety)

#### **Case Scenario:** 
>
> CoffeePro’s virtual coffee concierge has reached its final stages of development. With the model fine-tuned and equipped with Retrieval-Augmented Generation (RAG) capabilities, it’s time to ensure that the AI system performs effectively across various real-world scenarios while adhering to safety standards.
>
> As the AI developer, your role is to evaluate the model’s functionality and robustness against multiple criteria. This includes assessing the accuracy of coffee recommendations, the reliability of brewing advice, and the system’s ability to handle edge cases, such as incomplete or contradictory user inputs. Furthermore, the evaluation must ensure that the virtual agent aligns with ethical AI guidelines, avoiding biased recommendations or potentially unsafe brewing instructions.
>
> Management has also emphasized that the virtual concierge should operate within brand guidelines, maintaining a professional and friendly tone to reinforce customer trust and satisfaction.
> 
> Your Tasks:
>
> (a) Performance Testing:
> Test the virtual agent on a range of user inputs to ensure it delivers accurate and personalized coffee recommendations. Simulate scenarios for both beginner coffee drinkers and enthusiasts with advanced preferences.
>
> (b) Safety and Ethical Review:
> Evaluate the model for any outputs that could be misleading, unsafe, or biased. For example, verify that brewing instructions are accurate and practical, and recommendations are inclusive of a diverse range of preferences.
>
> By the end of this activity, you will have a clear understanding of how to use Python libraries and sample use cases to test model safety and reliability.

---

### Pre-requisites: 
- Load your virtual agent. Click here to open
    - Run the code cell below to import the Google GenerativeAI Python module and initialize our LLM instance

In [7]:
# Code Segment
import ipywidgets as widgets
from IPython.display import display

# Define the Python code you want users to copy
code_snippet = """
# Import Google GenerativeAI Python module
import google.generativeai as genai

# Define Gemini API key
genai.configure(api_key="YOUR_GEMINI_API_KEY")

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

# Specify model name and define system instruction
model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
  system_instruction="You are to serve as an AI virtual agent-coffee concierge for a company known as CoffeePro.\n    As a leading coffee retailer CoffeePro, aims to enhance their service of of selling wide\n    arrays coffee beans and blends from all around the world by providing personalized recommendations. \n\n    Given a user's preferences, such as:\n    * Drinking preference: Black or with milk/sugar\n    * Roast level: Light, medium, or dark\n    * Brew method: Espresso, pour over, cold brew, or French press\n    * Flavor profile: Fruity, nutty, chocolatey, or floral\n\n    You should:\n    1. Analyze the user's preferences and access your knowledge base of coffee beans to identify suitable options.\n    2. Provide detailed descriptions of recommended coffees, including their origin, flavor profile, and ideal brewing methods, based on the information provided from you in the injected prompts.\n    3. Offer personalized advice on brewing techniques, water temperature, and grind size to optimize the coffee experience.\n    4. Share interesting coffee facts and trivia to engage the user and foster a deeper appreciation for coffee.\n    5. Provide recommendations for food pairings that complement the coffee's flavor profile.\n    6. Answer questions about coffee history, roasting processes, and brewing techniques in a clear and informative manner.\n    7. Maintain a friendly and conversational tone to create a positive user experience. ",
)

# Acceptable past chat for reference
chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "Hello",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Hello there! Welcome to CoffeePro, your personal coffee concierge. I'm here to help you discover your perfect cup.  Tell me a little about your coffee preferences so I can recommend something you'll love.  Do you typically drink your coffee black, or with milk and/or sugar? What roast levels do you prefer? What's your go-to brewing method? And are there any particular flavor profiles you enjoy (fruity, nutty, chocolatey, floral, etc.)?  The more information you share, the better I can tailor my recommendations.\n",
      ],
    },
  ]
)
"""
# Create a TextArea widget to display the code
code_widget = widgets.Textarea(
    value=code_snippet,
    placeholder='Python code',
    description='Code:',
    disabled=True,  # Disable editing to make it read-only
    layout=widgets.Layout(width='1400px', height='250px')  # Adjust size as needed
)

# Display the widget
display(code_widget)

Textarea(value='\n# Import Google GenerativeAI Python module\nimport google.generativeai as genai\n\n# Define …

- Load your virtual agent (cont.)
    - Initialize RAG by running the code below:

In [8]:
# Define the Python code you want users to copy
code_snippet = """
import pandas as pd

# Load the Excel file
df = pd.read_excel("solution-practice-learning-activity-3/ailtk-fine-tuning-data.xls")

# Combine relevant columns into a single document per row
# Example: Assume 'Title' and 'Content' columns
corpus = df.apply(lambda row: f"{row['input']}. {row['output']}", axis=1).tolist()

def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def return_response(query, corpus, top_n=5):
    similarities = []
    
    # Calculate similarity for each document in the corpus
    for doc in corpus:
        similarity = jaccard_similarity(query, doc)
        similarities.append(similarity)
    
    # Get the indices of the top_n most similar documents
    top_n_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]
    
    # Return the top_n most similar documents
    top_n_documents = [corpus[i] for i in top_n_indices]
    
    return top_n_documents
"""
# Create a TextArea widget to display the code
code_widget = widgets.Textarea(
    value=code_snippet,
    placeholder='Python code',
    description='Code:',
    disabled=True,  # Disable editing to make it read-only
    layout=widgets.Layout(width='1400px', height='250px')  # Adjust size as needed
)

# Display the widget
display(code_widget)

Textarea(value='\nimport pandas as pd\n\n# Load the Excel file\ndf = pd.read_excel("solution-practice-learning…

- Load your virtual agent (cont.)
    - Run the code cell below to Define a function to find documents similar to the user's input, provide your LLM with an injected prompt, and receive a response

In [13]:
# Define the Python code you want users to copy
code_snippet = """
# Define a function to find documents similar to the user's input, 
# Provide LLM with an injected prompt, and receive response


def generate_response_with_injected_prompt(user_prompt, corpus, model):
# Generates a response using a model with injected prompt from RAG results.

# Parameters:
# - user_prompt (str): The user's input prompt (e.g., preferences for coffee).
# - corpus (list): The corpus of documents to search for similarities.
# - model (object): The model used to generate content based on the injected prompt.

    # Returns:
    # - str: The model-generated response based on the injected prompt.
    
    # RAG result on the user's input
    rag_result = return_response(user_prompt, corpus)
    
    # View five most similar documents from corpus according to jaccard similarity
    print(rag_result)
    
    # Append input to create an injected prompt
    injected_prompt = f"{user_prompt} {rag_result}"
    
    # Call your model and input the injected prompt
    response = model.generate_content(injected_prompt)
    
    # Return the response text
    return response.text

"""
# Create a TextArea widget to display the code
code_widget = widgets.Textarea(
    value=code_snippet,
    placeholder='Python code',
    description='Code:',
    disabled=True,  # Disable editing to make it read-only
    layout=widgets.Layout(width='1000px', height='250px')  # Adjust size as needed
)

# Display the widget
display(code_widget)

Textarea(value='\n# Define a function to find documents similar to the user\'s input, \n# Provide LLM with an …

In [14]:
# Define the Python code you want users to copy
code_snippet = """
# Test the function `generate_response_with_injected_prompt`

# Sample user input
user_prompt = "I like dark roast, espresso coffee. I prefer chocolate and rich flavors."

print(generate_response_with_injected_prompt(user_prompt, corpus, model))

"""
# Create a TextArea widget to display the code
code_widget = widgets.Textarea(
    value=code_snippet,
    placeholder='Python code',
    description='Code:',
    disabled=True,  # Disable editing to make it read-only
    layout=widgets.Layout(width='1000px', height='150px')  # Adjust size as needed
)

# Display the widget
display(code_widget)

Textarea(value='\n# Test the function `generate_response_with_injected_prompt`\n\n# Sample user input\nuser_pr…

(a) Performance Testing - Test the virtual on the use cases provided

In [ ]:
# Load sample use cases from xls file

# Pytest

(b) Safety and Ethical Review - Test the model for safety

In [ ]:
# Load ethics testing shit (Probably from HuggingFace)

# Pytest

Outro...

#### [ Back to Learning Instructions 5](../learning-instructions-5.ipynb)